## Implementing the BPE algorithm
The aim of this exercise is to implement the BPE tokenization algorithm. As a reminder, the principle consists in gathering the words or “tokens” that appear the most times in succession.

For example, if we consider the corpus containing the words in the following table (with the number of occurrences of each word):

| words | occurrence |
|------|-----------|
| voting | 2 |
| vote | 3 |
| slow | 1 |
| slowly | 2 |


And if the initial “tokens” are the letters of the alphabet, then the prefix “vo” will initially be added to the list of sub-words (tokens), cause the bigram "v" "o" occurs 5 times (2 + 3).

The steps involved in implementing the BPE algorithm are as follows:
1. Download a text corpus (here a wikipedia page)
2. Cut the text into words (using the “space” and “ponctuation” characters) and count the number of occurrences of each word.
3. Initialize the word dictionary with the initial tokens (letters of the alphabet)
4. Run BPE algorithm (learn vocabulary)
5. Test token decomposition on selected sentences (apply learned rules)


In the documents you will observe a lot of TODO in the code, replace it by your code.


### The class we will complete


At the end we will create an object (python) class as it follows :

```python
class Tokenizer:
    ''' 
        A class for our Tokenizer

        Methods
        -------
        fit(text_corpus: str) : None
            Fit the tokenizer based on the input text corpus 
        tokenize(text: str): List[str]
            Tokenize a text and return the list of token ids
        detokenize(tokens: List[str]): str
            From a list of token ids return the corresponding text
    '''

    def __init__(self, vocabulary_size: int = 500):
        ''' 
            Parameters
            ----------
            vocabulary_size : int
                The expected size of the vocabulary

        '''
        super().__init__()
        self.vs = vocabulary_size

    def fit(self, text_corpus: str):
        ''' Train the tokenizer on the provided text.

            Parameters
            ----------
            text_corpus : str
                The text used to train the model
        '''
        raise NotImplementedError


    def tokenize(self, text: str) -> List[str]:
        ''' Tokenize a text.

            Parameters
            ----------
            text : str
                The text to tokenize

            Returns
            -------
            List[str]
                The list of tokens
        '''
        raise NotImplementedError

    def detokenize(self, tokens : List[int]) -> str:
        ''' Reverse the tokenization.

            Parameters
            ----------
            tokens : List[str]
                A list of token ids

            Returns
            -------
            str
                The text corresponding to tokens
        '''
        raise NotImplementedError

```

## Requirement

**For this lab you only need the python standard library :D**

In [53]:
import re # the regex library
import json # read export json format

from typing import List # to specify the types in function def
from collections import Counter # a tools to counts unique occurences

from urllib.request import urlopen, Request

### Step 1: Download a corpus

For this lab we will consider a wikipedia page in french [Grèce antique](https://fr.wikipedia.org/w/api.php?format=json&action=query&prop=extracts&explaintext&redirects=1&titles=Gr%C3%A8ce_antique), but you are free to choose any content you want !

In [54]:


url_request  = 'https://fr.wikipedia.org/w/api.php?format=json&action=query&prop=extracts&explaintext&redirects=1&titles=Gr%C3%A8ce_antique'
wikipedia_request = Request(url_request)
wikipedia_request.add_header("User-Agent", "Course (thomas.gerald@lisn.fr)")
raw_page = urlopen(wikipedia_request)
json_page = json.load(raw_page)


In [55]:
corpus = list(json_page['query']['pages'].values())[0]['extract']
corpus[0:98]

"La Grèce antique est une civilisation de l'Antiquité des peuples de langue et de culture grecques "

### Step 2: Splitting text into words (or sequence of character no containing space)

To split the text into words, we'll use the following regex ```r'(\b[^\s]+\b)'```. To count words, we'll use python's Counter object. 
1. Store each word and its number of occurrences in **count_words**.
2. Give the 10 most frequent words (you'll store them in most_commons_words).

In [56]:

word_regex = re.compile(r'(\b[^\s]+\b)')
words = word_regex.findall(corpus)

count_words = Counter(words)
most_commons_words = count_words.most_common(10)
print(most_commons_words)

[('de', 1694), ('la', 1101), ('et', 978), ('des', 929), ('les', 773), ('à', 601), ('le', 519), ('en', 481), ('qui', 380), ('du', 352)]


### Step 3: Initialize word dictionary with initial tokens (letters of the alphabet)

Create the initial vocabulary in the vocab variable. How many initial tokens do you have?

In [57]:
vocab = {a for word in words for a in word}
vocab

{"'",
 '(',
 ')',
 ',',
 '-',
 '.',
 '/',
 '0',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 'A',
 'B',
 'C',
 'D',
 'E',
 'F',
 'G',
 'H',
 'I',
 'J',
 'K',
 'L',
 'M',
 'N',
 'O',
 'P',
 'Q',
 'R',
 'S',
 'T',
 'U',
 'V',
 'W',
 'X',
 'Y',
 'Z',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z',
 '²',
 'À',
 'Â',
 'É',
 'à',
 'â',
 'ç',
 'è',
 'é',
 'ê',
 'î',
 'ï',
 'ó',
 'ô',
 'ù',
 'û',
 'ü',
 'Œ',
 'œ',
 'Α',
 'Γ',
 'Ε',
 'Ι',
 'Ο',
 'Υ',
 'Φ',
 'Χ',
 'Ψ',
 'Ω',
 'α',
 'ι',
 'κ',
 'ρ',
 'ς',
 'ό',
 '’'}

In [58]:
len(vocab)

105

### Step 4: Learning the tokenizer
To learn the tokenizer we need several functions:
1. A function to calculate the frequency of each token pair.
2. A function to merge a pair

Several variables will be required:
1. **vocab** containing current vocabulary
2. **merge_rules** containing all the merge rules (a dictionary containing as key a pair of tokens to merge and the result of the token merge). For example: {('e', 's'), 'es', ('en', 't') :'ent'}.
3. **splits** A dictionary containing the current breakdown of the corpus, with the word as key and the list of “tokens” as value.


In [59]:
# In the first step splits contains the words broken down into characters

splits = {word: [char for char in word] for word in words}
splits

{'La': ['L', 'a'],
 'Grèce': ['G', 'r', 'è', 'c', 'e'],
 'antique': ['a', 'n', 't', 'i', 'q', 'u', 'e'],
 'est': ['e', 's', 't'],
 'une': ['u', 'n', 'e'],
 'civilisation': ['c', 'i', 'v', 'i', 'l', 'i', 's', 'a', 't', 'i', 'o', 'n'],
 'de': ['d', 'e'],
 "l'Antiquité": ['l', "'", 'A', 'n', 't', 'i', 'q', 'u', 'i', 't', 'é'],
 'des': ['d', 'e', 's'],
 'peuples': ['p', 'e', 'u', 'p', 'l', 'e', 's'],
 'langue': ['l', 'a', 'n', 'g', 'u', 'e'],
 'et': ['e', 't'],
 'culture': ['c', 'u', 'l', 't', 'u', 'r', 'e'],
 'grecques': ['g', 'r', 'e', 'c', 'q', 'u', 'e', 's'],
 'développée': ['d', 'é', 'v', 'e', 'l', 'o', 'p', 'p', 'é', 'e'],
 'en': ['e', 'n'],
 'dans': ['d', 'a', 'n', 's'],
 'la': ['l', 'a'],
 'partie': ['p', 'a', 'r', 't', 'i', 'e'],
 'occidentale': ['o', 'c', 'c', 'i', 'd', 'e', 'n', 't', 'a', 'l', 'e'],
 "l'Asie": ['l', "'", 'A', 's', 'i', 'e'],
 'Mineure': ['M', 'i', 'n', 'e', 'u', 'r', 'e'],
 'puis': ['p', 'u', 'i', 's'],
 'à': ['à'],
 'suite': ['s', 'u', 'i', 't', 'e'],
 'plusieu

#### Compute the frequency of token pairs
Create a function **compute_pair_freqs**, which, given the words broken down into tokens (splits dictionary) and the frequency of the words, returns the frequency of each pair of tokens (note only successive sub-words).

In [60]:
def compute_pair_freqs(splits, word_freqs):
    pair_freqs = {}
    for word, freq in word_freqs.items():
        for i in range(len(splits[word]) - 1):
            pair = (splits[word][i], splits[word][i+1])
            if pair in pair_freqs:
                pair_freqs[pair] += freq
            else:
                pair_freqs[pair] = freq

    return pair_freqs

In [61]:
pair_freqs = compute_pair_freqs(splits, count_words)
{k: pair_freqs[k] for k  in list(pair_freqs.keys())[:5]}

{('L', 'a'): 165,
 ('G', 'r'): 255,
 ('r', 'è'): 244,
 ('è', 'c'): 270,
 ('c', 'e'): 1163}

#### Find the most frequent pair and merge a pair
1. Create a function **most_frequent(pair_freqs)** returning the most frequent pair of tokens.
2. Create a **merge_pair()** function which, given a pair, returns the new splits of the corpus.

In [62]:
def most_frequent(pair_freqs):
    max_pair = max(pair_freqs, key=pair_freqs.get)
    return max_pair, pair_freqs[max_pair]
most_frequent(pair_freqs)

(('e', 's'), 5663)

In [63]:
def merge_pair(a : str, b: str, splits):
    '''
        splits : the dataset of words tokenized
        a : the first token
        b : the second tokens

        return : splits with a,b tokens merged
    '''
    for word in splits.keys():
        splits[word] = [token if token != a or token != b else a + b for token in splits[word]]

    return splits

In [64]:
new_splits = merge_pair(*most_frequent(pair_freqs)[0], splits)
print(new_splits['grecques'])

['g', 'r', 'e', 'c', 'q', 'u', 'e', 's']


#### Apply the algorithm until the desired vocabulary size is reached.
Create a BPE object that takes as arguments a corpus, a vocabulary size and train the BPE algorithm. The algorithm stores the final vocabulary in the **vocabulary** attribute and the merge rules in **merge_rules**.
For merge_rules, here's an example of its contents:
```
{('e', 's'): 'es',
 ('n', 't'): 'nt',
 ('q', 'u'): 'qu',
 ('r', 'e'): 're',
 ('o', 'n'): 'on',
 ('d', 'e'): 'de',
 ('l', 'e'): 'le',
 ('t', 'i'): 'ti',
 ('l', 'a'): 'la',
 ('i', 's'): 'is',
 ('e', 'nt'): 'ent', ...
 }
```

In [68]:
class BPE:
    def __init__(self, corpus, vocabulary_size=500):
        self.word_regex = re.compile(r"(\b[^\s]+\b)")
        words = self.word_regex.findall(corpus)

        # Count the frequency of each word in the corpus
        count_words = Counter(words)

        # Create initial vocab (set of all unique characters)
        self.vocab = list({char for word in count_words.keys() for char in word})
        self.vocab.sort()

        # Create the initial split: { "word": ['w', 'o', 'r', 'd'] }
        splits = {word: [c for c in word] for word in count_words.keys()}

        # Initialise merge_rules to track order of merges
        self.merge_rules = {}

        # --- TODO Section Implementation ---
        while len(self.vocab) < vocabulary_size:
            pair_counts = Counter()

            # 1. Count all adjacent pairs in the current splits
            # We must weight the count by the frequency of the word in the corpus
            for word, tokens in splits.items():
                freq = count_words[word]
                for i in range(len(tokens) - 1):
                    pair = (tokens[i], tokens[i + 1])
                    pair_counts[pair] += freq

            # If no pairs are found, stop (e.g., only single characters left)
            if not pair_counts:
                break

            # 2. Find the most frequent pair
            best_pair = pair_counts.most_common(1)[0][0]

            # 3. Create the new merged token
            new_token = best_pair[0] + best_pair[1]

            # 4. Add to vocabulary and record the rule
            self.vocab.append(new_token)
            self.merge_rules[best_pair] = new_token

            # 5. Update the splits dictionary with the new token
            for word in splits:
                token_list = splits[word]
                if len(token_list) < 2:
                    continue

                new_split = []
                i = 0
                while i < len(token_list):
                    # Check if current and next token match the best pair
                    if (
                        i < len(token_list) - 1
                        and (token_list[i], token_list[i + 1]) == best_pair
                    ):
                        new_split.append(new_token)
                        i += 2  # Skip the next token as it's merged
                    else:
                        new_split.append(token_list[i])
                        i += 1
                splits[word] = new_split
        # -----------------------------------

    def tokenize(self, text):
        words = self.word_regex.findall(text)
        splits = [[l for l in word] for word in words]

        # Apply merge rules in the order they were learned (Python dicts preserve insertion order)
        for pair, merge in self.merge_rules.items():
            for idx, split in enumerate(splits):
                i = 0
                while i < len(split) - 1:
                    if split[i] == pair[0] and split[i + 1] == pair[1]:
                        split = split[:i] + [merge] + split[i + 2 :]
                    else:
                        i += 1
                splits[idx] = split

        return sum(splits, [])

In [69]:
my_bpe = BPE(corpus)

In [77]:
texte = '''culture grecques développée en Grèce '''
token_my_bpe = my_bpe.tokenize(texte)[:12]
token_my_bpe

['culture', 'grecques', 'dévelop', 'p', 'ée', 'en', 'Grèce']

#### Test by modifying parameters or corpus
Test the algorithm with different hyper-parameters or data

## Using sentencepiece
We're now going to use the `sentencepiece` library, which can be installed (if not already installed) with pip : 

`! pip install sentencepiece`

To train the tokenizer, we'll use the `train` function of `SentencePieceTrainer`. 

In [72]:
! pip install sentencepiece

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 9.8 MB/s eta 0:00:00


In [73]:
import sentencepiece as spm

with open("test-cours.txt", "w") as f:
    f.write(corpus)
spm.SentencePieceTrainer.train(input="test-cours.txt", model_type='BPE',  model_prefix='m', vocab_size=500)

sentencepiece_trainer.cc(78) LOG(INFO) Starts training with : 
trainer_spec {
  input: test-cours.txt
  input_format: 
  model_prefix: m
  model_type: BPE
  vocab_size: 500
  self_test_sample_size: 0
  character_coverage: 0.9995
  input_sentence_size: 0
  shuffle_input_sentence: 1
  seed_sentencepiece_size: 1000000
  shrinking_factor: 0.75
  max_sentence_length: 4192
  num_threads: 16
  num_sub_iterations: 2
  max_sentencepiece_length: 16
  split_by_unicode_script: 1
  split_by_number: 1
  split_by_whitespace: 1
  split_digits: 0
  pretokenization_delimiter: 
  treat_whitespace_as_suffix: 0
  allow_whitespace_only_pieces: 0
  required_chars: 
  byte_fallback: 0
  vocabulary_output_piece_score: 1
  train_extremely_large_corpus: 0
  seed_sentencepieces_file: 
  hard_vocab_limit: 1
  use_all_vocab: 0
  unk_id: 0
  bos_id: 1
  eos_id: 2
  pad_id: -1
  unk_piece: <unk>
  bos_piece: <s>
  eos_piece: </s>
  pad_piece: <pad>
  unk_surface:  ⁇ 
  enable_differential_privacy: 0
  differential_pr

Looking at the “m.vocab” file, what are the differences with the vocabulary learned with your implementation? What modifications could you consider to obtain a similar vocabulary?

## Detokenization

Propose and implement a methods to detokenize encoded sentence (you should want to extend your vocabulary)

In [89]:
def detokenize(tokens):
    # 1. Join all subword tokens into a single string
    reconstructed = " ".join(tokens)

    # 2. Replace the EOW marker with a space to restore boundaries
    #    and strip leading/trailing whitespace
    return reconstructed.replace("###", " ").strip()

In [90]:
detokenize(token_my_bpe)

'culture grecques dévelop p ée en Grèce'